#### Start by creating a directory with the name of the desired country followed by a date (e.g., nigeria2016).

In [ ]:
import os

# Define the country name
country = 'nigeria2016'

# Check if the directory already exists
if not os.path.exists(country):
    # Create the directory if it doesn't exist
    os.makedirs(country)
    print(f"Directory '{country}' created.")
else:
    print(f"Directory '{country}' already exists.")

#### Create and add `acled.csv` file in the created directory (e.g., nigeria2016).

For more information, visit https://flee.readthedocs.io/en/master/Simulation_instance_construction/

#### This builder extracts cities/towns populations from an HTML file and stores it in the population.csv file. 

The desired country can be found by searching the https://www.citypopulation.de/ site (e.g., https://www.citypopulation.de/en/nigeria/cities/).

Store the webpage "CTRL+S" as an HTML file (e.g., population.html) and place in in the created directory.

The builder accesses the tables in HTML file and extract data from a specified table (e.g., table[0], table[1]). Specify the table in the code "line 19".

In [ ]:
import os
import pandas as pd

# This also becomes the name of the directory where files are stored.
country = 'nigeria2016'

html_file = '{}/population.html'.format(country, country)

if os.path.isfile(html_file):
    tables = pd.read_html(html_file)
    # Continue with further processing of the tables
else:
    print("The file '{}' is not found.".format(html_file))

# Uncomment to make sure the tables are accessed from the html file
# print(tables)

# Specify table with major cities' names and population
table = tables[0]

# Uncomment to make sure the desired table is accessed from tables
# print(table)

# Please make sure the column names exist in the specified table
selected_columns = table[['Name', 'Population Census (Cf) 2006-03-21']]

# Drop rows with missing values in the selected columns
selected_columns = selected_columns.dropna()

# Rename columns
selected_columns.columns = ['name', 'population']

# Filter rows with population greater than 10,000
selected_columns = selected_columns[selected_columns['population'] > 10000]

# Save the data to a CSV file
output_file = '{}/population.csv'.format(country)
selected_columns.to_csv(output_file, index=False)

print(f'{country}/population.csv created. Please inspect the file for non-standard characters!')

#### This builder extracts locations using acled.csv and created population.csv files, and stores them in locations.csv file. 

Set the conflict_threshold to only assign conflict zones to location types with conflict periods greater than the threshold.

In [ ]:
import pandas as pd
import numpy as np
import warnings
import sys
import os
import calendar as cal
from datetime import datetime
import json
import requests
import time
import wikipedia
import wbdata

def date_format(in_date):
    # converting date from textbased to dd-mm-yyyy format
    if "-" in in_date:
        split_date = in_date.split("-")
    else:
        split_date = in_date.split(" ")

    month_num = month_convert(split_date[1])
    if int(split_date[2]) < 50:
        year = int(split_date[2]) + 2000
    else:
        year = int(split_date[2])
    out_date = split_date[0] + "-" + str(month_num) + "-" + str(year)
    return out_date

def month_convert(month_name):
    months = {
    "jan": "01", "january": "01",
    "feb": "02", "february": "02",
    "mar": "03", "march": "03",
    "apr": "04", "april": "04",
    "may": "05", "may": "05",
    "jun": "06", "june": "06",
    "jul": "07", "july": "07",
    "aug": "08", "august": "08",
    "sep": "09", "september": "09",
    "oct": "10", "october": "10",
    "nov": "11", "november": "11",
    "dec": "12", "december": "12"
    }

    # Convert the month name to lowercase and strip leading/trailing whitespace
    month_name = month_name.strip().lower()

    # Look up the month number in the dictionary
    if month_name in months:
        month_num = months[month_name]
        #print(f"The month number for {month_name} is {month_num}.")
    else:
        print("Invalid month name entered.")

    return month_num

def between_date(d1, d2):
    # Gets difference between two dates in string format "dd-mm-yyyy"
    d1list = d1.split("-")
    d2list = d2.split("-")
    date1 = datetime(int(d1list[2]), int(d1list[1]), int(d1list[0]))
    date2 = datetime(int(d2list[2]), int(d2list[1]), int(d2list[0]))

    return abs((date1 - date2).days)  # Maybe add +1

def drop_rows(inputdata, columnname, dropparameter):
    removedrows = inputdata.index[
        inputdata[columnname] <= dropparameter].tolist()
    outputdata = inputdata.drop(removedrows)
    return outputdata

def get_state_population(state_name, population_input_file):
    df = pd.read_csv(population_input_file)
    filtered_df = df[df['name'] == state_name]
    if len(filtered_df) > 0:
        population = filtered_df['population'].values[0]
        return population
    else:
        return None


def get_city_population(city_name,population_input_file):
    country_code = 'ML'
    url = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities/{0}".format(get_wikidata_id(city_name))

    headers = {
        "X-RapidAPI-Key": "6e1b07b54fmsh14df87e58d9db7bp175272jsn85fd0398365f",
        "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
    }
    response = requests.request("GET", url, headers=headers)

    if response.status_code == 404:
        get_state_population(city_name,population_input_file)

    else:
        data = response.json()
        population = data["data"]['population']
        return population
    
def filter_table(df, colname, adminlevel):
    if adminlevel == "admin1":
        adminlist = df.admin1.unique()
    elif adminlevel == "location":
        adminlist = df.location.unique()
    else:
        adminlist = df.admin2.unique()

    newdf = pd.DataFrame()  

    for admin in adminlist:
        tempdf = df.loc[df[adminlevel] == admin]
        tempdf.sort_values(colname, ascending=True)
        newdf = pd.concat([newdf, tempdf.tail(1)])

    return newdf


def acled2locations(country, start_date, filter_opt, location_type="admin1", conflict_threshold=100):
    current_dir = os.getcwd()
    input_file = os.path.join(current_dir, country, "acled.csv") 
    
    try:
        df = pd.read_csv(input_file)
    except:
        print("Runtime Error: File Cannot be found")
        return
    
    df = df[["event_date", "country", "admin1", "admin2", "location", "latitude", "longitude", "event_type", "sub_event_type", "fatalities"]]
    
    event_dates = df["event_date"].tolist()
    
    formatted_event_dates = [date_format(date) for date in event_dates]
    
    # Calculate the number of days between each event's date and the start_date
    conflict_dates = [between_date(d, start_date) for d in formatted_event_dates]
    
    # Add days to the df
    df['conflict_date'] = conflict_dates
    
    # Including all events
    fatalities_threshold = 0
    
    # Dropping rows based on the 'fatalities_threshold'    
    df = drop_rows(df, 'fatalities', fatalities_threshold)
    
    # Sorting the df by 'conflict_date' and 'admin1' and drops duplicate rows
    df = df.sort_values(["conflict_date", "admin1"]).drop_duplicates(["conflict_date", "admin1"])
    
    if filter_opt == 'earliest':
        filter_opt = 'conflict_date'

    try:
        df = filter_table(df, filter_opt, location_type)
    except:
        print("Runtime error: filter_opt value must be earliest or fatalities")
        
    output_df = df[['admin1', 'country', 'latitude', 'longitude', 'conflict_date']]
    
    output_df.columns = ['name', 'country', 'latitude', 'longitude', 'conflict_date'] 
    
    # Create two DataFrames for towns and conflict zones
    towns_df = df[df['conflict_date'] <= conflict_threshold].copy()
    conflict_zones_df = df[df['conflict_date'] > conflict_threshold].copy()
    
    # Assign location types
    towns_df['location_type'] = 'town'
    conflict_zones_df['location_type'] = 'conflict_zone'
    
    # Merge the DataFrames back together if necessary
    merged_df = pd.concat([towns_df, conflict_zones_df])
    
    # Create a dictionary to store location names from population.csv as keys and their populations as values
    population_dict = {row['name']: row['population'] for index, row in population_df.iterrows()}
    
    # Add population to the merged_df based on 'admin1' names
    merged_df['population'] = [population_dict.get(name, 0) for name in merged_df['admin1']]
    
    # Replace any infinite or NaN values with 0 and convert population to integers
    merged_df['population'] = merged_df['population'].replace([np.inf, -np.inf, np.nan], 0)
    merged_df['population'] = merged_df['population'].astype(int)
    
    # Reorder columns and rename them
    merged_df = merged_df[['admin1', 'country', 'latitude', 'longitude', 'location_type', 'conflict_date', 'population']]
    merged_df.columns = ['name', 'country', 'latitude', 'longitude', 'location_type', 'conflict_date', 'population']

    # Save the final 'locations.csv' file
    output_file = os.path.join(country, "locations.csv")
    merged_df.to_csv(output_file, index=False)

    print(f'{country}/locations.csv created. Please inspect the file for unwanted anomalies!')

    
# Please replace the country name 
country = "nigeria2016"
start_date = "01-01-2016"
filter_opt = 'earliest'
location_type = "admin1"
conflict_threshold = 100

population_input_file = os.path.join(country, "population.csv")

acled2locations(country, start_date, filter_opt, location_type="admin1", conflict_threshold=100)

#### This builder constructs conflicts.csv from created locations.csv file.

In [ ]:
import os
import sys
import csv

# Replace path with the actual directory path
flee_path = '/home/mghorbani/workspace/flee'
sys.path.append(flee_path)

# Now you can import the modules from the 'flee' package
from flee.InputGeography import InputGeography

def find_column_index(header, column_name):
    # Find the index of a column by matching its name in the header
    for i, col in enumerate(header):
        if col == column_name:
            return i
    return -1

def location2conflict(simulation_period, input_file, output_file):
    ig = InputGeography()
    ig.ReadLocationsFromCSV(input_file)

    with open(input_file, "r") as file:
        reader = csv.reader(file)
        header = next(reader)
        conflict_zone_index = find_column_index(header, "location_type")
        conflict_date_index = find_column_index(header, "conflict_date")

    # Create the header string for the output file without the "name" column
    output_header_string = "day"
    for l in ig.locations:
        if l[conflict_zone_index] == "conflict_zone":
            output_header_string += ",%s" % l[0]
    output_header_string += "\n"

    with open(output_file, "w") as file:
        file.write(output_header_string)

        for t in range(0, simulation_period):
            output = "%s" % t
            for l in ig.locations:
                if l[conflict_zone_index] == "conflict_zone":
                    confl_date = int(l[conflict_date_index])
                    if confl_date <= t:
                        output += ",1"
                    else:
                        output += ",0"
            output += "\n"
            file.write(output)


# Set the values for simulation_period, input_file, and output_file
simulation_period = 365

# Please specify a country name
country = "nigeria2016"

input_file = os.path.join(country, "locations.csv")

output_file = os.path.join(country, "conflicts.csv")

# Call the function location2conflict
location2conflict(simulation_period, input_file, output_file)

print(f'{country}/conflicts.csv created.')

####  This builder simulates forcasting conflict data in a given country and visualising it using custom distribution functions. 

In [ ]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from datetime import date

# Read 
def read_conflict_zones(filename):
    try:
        df = pd.read_csv(filename)
        conflict_zones = df.columns[1:].tolist()
        conflict_zones = [zone for zone in conflict_zones if zone]  # Exclude empty headers
        return conflict_zones
    except FileNotFoundError:
        print("File not found: ", filename)
        return []
    
    
# Custom function to generate all zeros csv file
def generate_conflict_zones_csv(filename, conflict_zones, period):
    data = {'Days': list(range(period))}
    data.update({zone: [0] * period for zone in conflict_zones})
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    

# Custom distribution function
def custom_distribution(intensity, x):
    max_value = intensity
    peak_day = 365
    std_deviation = 800
    variation_factor = 0.01  # Adjust the variation factor as desired
    
    spreading_factor = np.exp(-((x - peak_day) / std_deviation) ** 2)
    
    # Add random fluctuations to the spreading factor
    spreading_factor += np.random.normal(0, variation_factor, len(x))
    
    y = max_value * spreading_factor
    return y

# Specify the simulation country
country = 'nigeria2016'

# Create the path to input file
input_file = os.path.join(country, 'conflicts.csv')

# Read the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Call the function to extract the conflict_zones from the file
conflict_zones = read_conflict_zones(input_file)

# Set the intensity as maximum number of conflict zones 
intensity = len(conflict_zones)

# Generate x-axis values
start_date = date(2016, 1, 1)
current_date = date(2016, 12, 31)
days_passed = (current_date - start_date).days

# Specify the simulation period for forcasting
period = days_passed * 2

# Generate x-axis values using a custom_distributions
x1 = np.linspace(0, days_passed, num=days_passed).astype(int)
x2 = np.linspace(days_passed, period, num=period - days_passed).astype(int)

# Generate y-axis values using conflicts values
y1 = df.iloc[:, 1:days_passed + 1].sum(axis=1)[:days_passed]

# Generate y-axis values using a custom_distribution
y2 = custom_distribution(intensity, x2)

# Combine the generated data
x = np.concatenate((x1, x2))
y = np.concatenate((y1, y2))

# Create subplots
fig, (ax1, ax2) = plt.subplots(1, 2)

# Plot graph
ax1.plot(x, y)
ax1.set_title('Custom Distribution')
ax1.set_xlabel('Days')
ax1.set_ylabel('Conflict Zones')

# Convert y values to integers
y = [int(val) for val in y]

# Create path to modified CSV file
output_file = os.path.join(country, "simulated-conflicts.csv")

# Call the function to generate all zeros csv file
generate_conflict_zones_csv(output_file, conflict_zones, period)

# Read the CSV file into a DataFrame
df_modified = pd.read_csv(output_file)

# Update the modified DataFrame with the new y values
modified_rows = []
y_index = 0
for _, row in df_modified.iterrows():
    row = row.values
    if y_index >= len(y):
        break

    number = y[y_index]
    y_index += 1
    assigned_count = 0
    for i in range(1, len(row)):
        if assigned_count < number:
            row[i] = 1
            assigned_count += 1
        else:
            row[i] = 0

    modified_rows.append(row)
    
# Create modified DataFrame
modified_df = pd.DataFrame(modified_rows, columns=df.columns)

# Compute the sum of each row (excluding the '#Day' column)
sum_values = modified_df.iloc[:, 1:].sum(axis=1)

# Plot the summed values
ax2.plot(x, sum_values)
ax2.set_title('simulated-conflicts.csv')
ax2.set_xlabel('Days')

# Save the modified DataFrame to the output file
modified_df.to_csv(output_file, index=False)

print(f'{country}/simulated-conflicts.csv created.')